## Merging data sets

### Table of Contents
1. [Initial exploration](#exploration)
2. [Merging datasets](#merging)
3. [Saving merged dataset](#saving)

### Initial exploration <a class="anchor" id="exploration"></a>

Adding more features to a dataset helps uncover hidden patterns and improve model performance. By including additional relevant features, we get more context and can better understand the data. 

In [1]:
import pandas as pd
import numpy as np

Let's import all the datasets

In [2]:
train = pd.read_csv("../data/train.csv")
stores = pd.read_csv("../data/stores.csv")
holidays = pd.read_csv("../data/holidays_events.csv")
oil = pd.read_csv("../data/oil.csv")
transactions = pd.read_csv("../data/transactions.csv")
test = pd.read_csv("../data/test.csv")

Our main goal is to train the model and use it to predict sales values on the test set. For accurate sales predictions, the test set must have the same features as the training set. So, we should combine both sets, add features, do some exploratory data analysis (EDA), and then split the data again before feature engineering.

In [3]:
df = pd.concat([train, test], axis=0, ignore_index=True)

#### Oil dataset
Let's check for missing values in 'oil' dataframe before merging. 

In [7]:
oil['dcoilwtico'].isnull().sum()

43

There are 43 missing values. Let's find out how big is that in terms of percentage from the total number of values in our dataset.

In [8]:
missing_values = oil['dcoilwtico'].isnull().sum()
percentage_missing = (missing_values / oil.shape[0]) * 100

print(f'Percentage of missing values for each column: {percentage_missing:.2f}%')

Percentage of missing values for each column: 3.53%


Since the number of missing values is less than 5% we can fill missing values with the next value in the column, which is useful for time series data.

In [9]:
oil['dcoilwtico'] = oil['dcoilwtico'].bfill()
# verifying results
oil['dcoilwtico'].isnull().sum()

0

#### Holidays_events dataset

Let's check for unique values in each categorical column

In [10]:
exclude_column = 'date'

for column in holidays.columns:
    if column != exclude_column:
        unique_values = holidays[column].unique()
        print(f'Unique values in column "{column}": {unique_values}')
    

Unique values in column "type": ['Holiday' 'Transfer' 'Additional' 'Bridge' 'Work Day' 'Event']
Unique values in column "locale": ['Local' 'Regional' 'National']
Unique values in column "locale_name": ['Manta' 'Cotopaxi' 'Cuenca' 'Libertad' 'Riobamba' 'Puyo' 'Guaranda'
 'Imbabura' 'Latacunga' 'Machala' 'Santo Domingo' 'El Carmen' 'Cayambe'
 'Esmeraldas' 'Ecuador' 'Ambato' 'Ibarra' 'Quevedo'
 'Santo Domingo de los Tsachilas' 'Santa Elena' 'Quito' 'Loja' 'Salinas'
 'Guayaquil']
Unique values in column "description": ['Fundacion de Manta' 'Provincializacion de Cotopaxi'
 'Fundacion de Cuenca' 'Cantonizacion de Libertad'
 'Cantonizacion de Riobamba' 'Cantonizacion del Puyo'
 'Cantonizacion de Guaranda' 'Provincializacion de Imbabura'
 'Cantonizacion de Latacunga' 'Fundacion de Machala'
 'Fundacion de Santo Domingo' 'Cantonizacion de El Carmen'
 'Cantonizacion de Cayambe' 'Fundacion de Esmeraldas'
 'Primer Grito de Independencia' 'Fundacion de Riobamba'
 'Fundacion de Ambato' 'Fundacion de 

Let's remove rows with working days, as they go against our purpose.

In [11]:
holidays = holidays[holidays['type'] != "Work Day"]
holidays.info()

<class 'pandas.core.frame.DataFrame'>
Index: 345 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         345 non-null    object
 1   type         345 non-null    object
 2   locale       345 non-null    object
 3   locale_name  345 non-null    object
 4   description  345 non-null    object
 5   transferred  345 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 16.5+ KB


Upon inspection, we find values labeled 'Event' in the 'type' column. Based on their descriptions, these events do not seem to be holidays but rather major sporting events or natural disasters. These events may influence sales patterns; therefore, we should create a separate column for them.

In [12]:
holidays['events'] = holidays['type'].apply(lambda x: x if x == 'Event' else None)
# replacing 'Event' value in 'type'  column with NaN value
holidays['type'].replace('Event', np.nan, inplace=True)

C:\Users\nicko\AppData\Local\Temp\ipykernel_5556\3349347922.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  holidays['type'].replace('Event', np.nan, inplace=True)


In [13]:
# verifying the results with a particular date which falls under 'Event' and 'Holiday'
holidays[holidays['date'] == '2013-05-12'].head(10)

,date,type,locale,locale_name,description,transferred,events
54,2013-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False,None
55,2013-05-12,NaN,National,Ecuador,Dia de la Madre,False,Event


In [14]:
# creating new column called 'holiday' containing boolean values
holidays['holiday'] = holidays['type'].notna()

In [15]:
#verifying results
holidays.info()

<class 'pandas.core.frame.DataFrame'>
Index: 345 entries, 0 to 349
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         345 non-null    object
 1   type         289 non-null    object
 2   locale       345 non-null    object
 3   locale_name  345 non-null    object
 4   description  345 non-null    object
 5   transferred  345 non-null    bool  
 6   events       56 non-null     object
 7   holiday      345 non-null    bool  
dtypes: bool(2), object(6)
memory usage: 19.5+ KB


Before merging holidays_events dataset it makes sence to split it into three parts based on the type of holidays: National, State and City.

In [16]:
national_holidays = holidays[(holidays['locale'] != "Local") & (holidays['locale'] != "Regional")]
# renaming 'type' column 
national_holidays = national_holidays.rename(columns={'holiday': 'national_holiday'})
# verifying results
national_holidays['locale'].unique()

array(['National'], dtype=object)

In [17]:
national_holidays.head(10)

,date,type,locale,locale_name,description,transferred,events,national_holiday
14,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False,None,True
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True,None,True
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False,None,True
21,2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False,None,True
22,2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False,None,True
31,2012-12-21,Additional,National,Ecuador,Navidad-4,False,None,True
33,2012-12-22,Additional,National,Ecuador,Navidad-3,False,None,True
34,2012-12-23,Additional,National,Ecuador,Navidad-2,False,None,True
35,2012-12-24,Bridge,National,Ecuador,Puente Navidad,False,None,True
36,2012-12-24,Additional,National,Ecuador,Navidad-1,False,None,True


In [18]:
state_holidays = holidays[(holidays['locale'] != "Local") & (holidays['locale'] != "National")]
state_holidays = state_holidays.rename(columns={'holiday': 'state_holiday'})
state_holidays['locale'].unique()

array(['Regional'], dtype=object)

In [19]:
city_holidays = holidays[(holidays['locale'] != "National") & (holidays['locale'] != "Regional")]
city_holidays = city_holidays.rename(columns={'holiday': 'city_holiday'})
city_holidays['locale'].unique()

array(['Local'], dtype=object)

### Merging datasets <a id="merging"></a>

In [20]:
df_stores = df.merge(stores, left_on='store_nbr', right_on='store_nbr', how='left')
# verifying results
df_stores.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13


In [21]:
df_stores_oil = df_stores.merge(oil, left_on='date', right_on='date', how='left')
df_stores_oil.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14


In [22]:
df_stores_oil_trans = df_stores_oil.merge(transactions, left_on=['date','store_nbr'], right_on=['date','store_nbr'], how='left')
df_stores_oil_trans.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,NaN


In [23]:
# verifying results with a particular date
df_stores_oil_trans[df_stores_oil_trans['date'] == '2013-01-02'].head(10)

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,transactions
1782,1782,2013-01-02,1,AUTOMOTIVE,2.000,0,Quito,Pichincha,D,13,93.14,2111.0
1783,1783,2013-01-02,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,93.14,2111.0
1784,1784,2013-01-02,1,BEAUTY,2.000,0,Quito,Pichincha,D,13,93.14,2111.0
1785,1785,2013-01-02,1,BEVERAGES,1091.000,0,Quito,Pichincha,D,13,93.14,2111.0
1786,1786,2013-01-02,1,BOOKS,0.000,0,Quito,Pichincha,D,13,93.14,2111.0
1787,1787,2013-01-02,1,BREAD/BAKERY,470.652,0,Quito,Pichincha,D,13,93.14,2111.0
1788,1788,2013-01-02,1,CELEBRATION,0.000,0,Quito,Pichincha,D,13,93.14,2111.0
1789,1789,2013-01-02,1,CLEANING,1060.000,0,Quito,Pichincha,D,13,93.14,2111.0
1790,1790,2013-01-02,1,DAIRY,579.000,0,Quito,Pichincha,D,13,93.14,2111.0
1791,1791,2013-01-02,1,DELI,164.069,0,Quito,Pichincha,D,13,93.14,2111.0


In [24]:
df_stores_oil_trans_national_holidays = df_stores_oil_trans.merge(
    national_holidays[['national_holiday', 'date', 'events']], left_on=['date'], right_on= ['date'], how='left'
    )

df_stores_oil_trans_national_holidays.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,transactions,national_holiday,events
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None


In [25]:
df_stores_oil_trans_national_state_holidays = df_stores_oil_trans_national_holidays.merge(
    state_holidays[['state_holiday', 'date', 'locale_name']],left_on=['date','state'], right_on= ['date', 'locale_name'], how='left'
    ).drop('locale_name', axis=1)

df_stores_oil_trans_national_state_holidays.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,transactions,national_holiday,events,state_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN


In [26]:
df_final = df_stores_oil_trans_national_state_holidays.merge(
    city_holidays[['city_holiday', 'date', 'locale_name']],left_on=['date','city'], right_on= ['date', 'locale_name'], how='left'
    ).drop('locale_name', axis=1)

df_final.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,transactions,national_holiday,events,state_holiday,city_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN,NaN


In [27]:
# rename 'dcoilwtico' column into more intuitive name
df_final= df_final.rename(columns={'dcoilwtico': 'oil_price'})
df_final.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,oil_price,transactions,national_holiday,events,state_holiday,city_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,NaN,True,None,NaN,NaN


In [28]:
# filling blank values with boolean False
pd.set_option('future.no_silent_downcasting', True)
df_final = df_final.fillna({'national_holiday': False, 'state_holiday': False, 'city_holiday': False})

In [29]:
# converting 'event' column to boolean
df_final['events'] = df_final['events'].notna()

### Saving merged dataset <a id="saving"></a>

In [30]:
df_final.to_csv('../data/train_merged.csv', index = False)